데이터 크롤링 프로젝트: 서론/본론/결론이 있도록 작성
    
- 데이터 수집의 개요
- 수집 동기, 목적
- 데이터 수집의 계획 및 주기 작성
- 데이터를 수집하는 시스템의 구조도 및 프로세스 등을 설명
- 크롤링하는 방법
-  크롤링 코드는 서버에서 실행될 수 있도록 작성(local에서 하면 안되고 서버에서)
- 모듈이나 패키지 형태(.py)로 만들거나 스크래피 프레임워크를 사용
- 데이터의 저장
- 스크래피: mongodb 데이터베이스에 크롤링한 데이터를 저장
- 모듈, 패키지: mysql로 해도 됨.
- 코드의 관리
- github으로 코드 관리, 공유, 이슈처리
- 설치되어야하는 패키지는 requirements.txt로 작성
- 코드의 설명은 처음 접한 사람도 그대로 따라하면 코드를 실행할 수 있을 정도로 간결하게 작성
- 프로젝트 회고
- 프로젝트하면서 느낀점
- 추후 연구 및 개선 사항

In [1]:
import requests
import time
import scrapy

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from scrapy.http import TextResponse

In [8]:
!scrapy startproject youtube_review

Error: Module 'youtube_review' already exists


#### 유투브 첫 봐면에서 '타다금지법' 검색하여, 이번 주에 업로드된 동영상 제목, url 가져오기

In [2]:
# 링크
url = "https://www.youtube.com/"
driver=webdriver.Chrome()
driver.get(url)

req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [3]:
driver.find_element_by_xpath('//*[@id="search"]').send_keys("타다금지법")
driver.find_element_by_xpath('//*[@id="search-icon-legacy"]').click()

In [4]:
# 이번주 필터링
driver.find_element_by_xpath('//*[@id="container"]/ytd-toggle-button-renderer/a').click()

In [17]:
# title, link 가져오기
titles=driver.find_elements_by_xpath('//*[@id="video-title"]')
len(titles)

61

In [19]:
titles

[<selenium.webdriver.remote.webelement.WebElement (session="c21cb78175e96054d3a8a2faa98fdb14", element="3d5c1f5e-5c76-4dbd-96f6-82aa980d05e3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c21cb78175e96054d3a8a2faa98fdb14", element="fd1a10a4-0d9c-4e70-9858-4fcd5d07217a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c21cb78175e96054d3a8a2faa98fdb14", element="eb4eeeb1-fb47-49bd-8e22-26131dec0b1c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c21cb78175e96054d3a8a2faa98fdb14", element="48a9d2d6-ade1-44d1-a211-776b6e20b908")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c21cb78175e96054d3a8a2faa98fdb14", element="1b71c41b-1751-4a72-a74e-ff7f168040c2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c21cb78175e96054d3a8a2faa98fdb14", element="78c0ada2-1729-4f67-8b8b-26c756e9294a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c21cb78175e96054d3a8a2faa98fdb14", element="53d11a4b-2cae-4ece-8fed-de

In [18]:
link=driver.find_elements_by_xpath('//*[contains (@href, "/watch?")]')
link[1].get_attribute('href')

'https://www.youtube.com/watch?v=cKvVgFjOmGA'

In [ ]:
#상세 링크 접속하여 user_id, comment 가져오기

In [23]:
url = link[1].get_attribute('href')
driver=webdriver.Chrome()
driver.get(url)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=60613): Max retries exceeded with url: /session/ccfcd0411f03db30675f8cc2fbe4b64a/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022FFAEFB470>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [208]:
body = driver.find_element_by_tag_name('body')

num_of_pagedowns = 20

while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    num_of_pagedowns -= 1

In [1]:
#driver.find_element_by_xpath('//[@class="dropdown-trigger style-scope yt-dropdown-menu"]').click()

#driver.find_element_by_css_selector('#icon-label').click()



In [2]:
# 정렬 기준버튼 클릭

#driver.find_element_by_xpath('//*[@id="icon-label"]').click()

#인기순 정렬
#driver.find_element_by_xpath('//*[@id="menu"]/a[1]/paper-item/paper-item-body/div[1]').click

#최근날짜 순 정렬
#driver.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[1]').click()

In [6]:
comments = driver.find_elements_by_css_selector('#content-text')

In [7]:
comments[2].text

'국회의원 꼬라지 봐서는 택시들 승차거부 과속 난폭운전은 영원히 사라지지 않겠내 ㅋㅋㅋ\n역시 이나라는 선거전에 표걸고 떼법 밀어붙이는게 최고여 ㅋㅋㅋ'

In [28]:
user_id = driver.find_elements_by_css_selector('#author-text > span')
len(user_id)

20

In [29]:
user_id[0].text

'미스터손'

In [32]:
datas = [] #제목

for i in range(len(comments)):
    comment = comments[i].text
    user_id = user_id[i].text
    
    datas.append({
    "name" : user_id,
    "comment" : comment
        })

pd.DataFrame(datas)

,name,comment
0,미스터손,"내 기억속의 택시는 기본요금 나왔다고 카드로 결제한다고 얼굴 찌푸렸던 택시, 물웅덩..."
1,모모,난택시가 싢다 왜냐하면 좋은택시 기사도있었지만 난폭운전택시기사 끼어들기 u턴...
2,조창,국회의원 꼬라지 봐서는 택시들 승차거부 과속 난폭운전은 영원히 사라지지 않겠내 ㅋㅋ...
3,김영선,와이티엔은 타다편향 방송 안하고 공정하게 양측 울화통 터지지 않게 겸손한 방송 엄...
4,Frienze,중국애들 몰려드네
5,레전드마스터,"면허사서 합법적으로 할생각을해야지!!!\n이게4차산업이러면 5차산업은 \n판사,검사..."
6,에비스자와쿠루미,타다서비스는 엄연히 법의 허점을 이용한 불법적인 사업인데 이걸 없애는걸 이상하게 생...
7,정담원,택시들 밥그릇지키기\n국민들은 다 알고있다
8,포푸라,부산만 하더라도 택시가 24000대인데\n무분별하게 타다같은 업체가 렌탈카니발 증차...
9,부엉이오형제,국민의 다수가 원한다고 불법을 지양할순없는겁니다. 타다는 애당초 무허가콜택시사업이었...


### Scrapy Code 작성
- items.py
- spider.py
- run.sh

In [3]:
!cat items.py

import scrapy

class CrawlerItem(scrapy.Item):
    title = scrapy.Field()
    s_price=scrapy.Field()
    o_price=scrapy.Field()
    link=scrapy.Field()


In [ ]:
# items.py

In [ ]:
%%writefile items.py
import scrapy


class CrawlerItem(scrapy.Item):
    title = scrapy.Field()
    link = scrapy.Field()
    user_id = scrapy.Field()
    comment = scrapy.Field()

In [ ]:
# spider.py

In [ ]:
# run.sh 실행

In [ ]:
# 출력되는 데이터를 csv 파일로 저장

In [3]:
# mongo DB로 보내기